In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import os
from torch.autograd import Variable
from torchvision import datasets, models, transforms


In [2]:

data_transforms = {
    'Train_Alphabet': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test_Alphabet': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



In [7]:
data_dir = 'C:/Sign Language Detection/Module 1 (sign to text)/processed_dataset/'



In [8]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['Train_Alphabet', 'Test_Alphabet']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['Train_Alphabet', 'Test_Alphabet']}



In [9]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['Train_Alphabet', 'Test_Alphabet']}
class_names = image_datasets['Train_Alphabet'].classes


In [10]:
len(class_names)

26

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [12]:
device

device(type='cuda', index=0)

In [13]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 26)


c:\Users\muham\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\muham\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)



In [15]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['Train_Alphabet', 'Test_Alphabet']:
            if phase == 'Train_Alphabet':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'Train_Alphabet'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    loss = criterion(outputs, labels)

                    if phase == 'Train_Alphabet':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'Train_Alphabet':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

    return model



In [17]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)


Epoch 0/9
----------
Train_Alphabet Loss: 1.1372 Acc: 0.6703
Test_Alphabet Loss: 0.0697 Acc: 0.9782
Epoch 1/9
----------
Train_Alphabet Loss: 0.4997 Acc: 0.8571
Test_Alphabet Loss: 0.0251 Acc: 0.9938
Epoch 2/9
----------
Train_Alphabet Loss: 0.3671 Acc: 0.8942
Test_Alphabet Loss: 0.0121 Acc: 0.9961
Epoch 3/9
----------
Train_Alphabet Loss: 0.2906 Acc: 0.9174
Test_Alphabet Loss: 0.0127 Acc: 0.9973
Epoch 4/9
----------
Train_Alphabet Loss: 0.2480 Acc: 0.9261
Test_Alphabet Loss: 0.0050 Acc: 0.9980
Epoch 5/9
----------
Train_Alphabet Loss: 0.2190 Acc: 0.9357
Test_Alphabet Loss: 0.0075 Acc: 0.9977
Epoch 6/9
----------
Train_Alphabet Loss: 0.1881 Acc: 0.9464
Test_Alphabet Loss: 0.0025 Acc: 0.9988
Epoch 7/9
----------
Train_Alphabet Loss: 0.1276 Acc: 0.9634
Test_Alphabet Loss: 0.0002 Acc: 1.0000
Epoch 8/9
----------
Train_Alphabet Loss: 0.1107 Acc: 0.9684
Test_Alphabet Loss: 0.0001 Acc: 1.0000
Epoch 9/9
----------
Train_Alphabet Loss: 0.0902 Acc: 0.9739
Test_Alphabet Loss: 0.0001 Acc: 1.0000


In [18]:
torch.save(model, 'signlang_model.pth')
